In [4]:
import pandas as pd

In [5]:
import matplotlib.pyplot as plt

In [6]:
import numpy as np

In [7]:
df=pd.read_csv('data.csv')

In [8]:
#df.head()

In [9]:
#df.tail()

In [10]:
df1=df.reset_index()[' PREV_CLOSE']

In [11]:
#df1.shape #gives number of enties in "PREV_CLOSE" Column

In [12]:
#df1

In [13]:
#plt.plot(df1) #plotting prev_close price

In [14]:
#LSTM model is sensitive to data so coverting data in range :0 to 1
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [15]:
df1

array([[0.00100048],
       [0.00024175],
       [0.02146071],
       ...,
       [0.00156549],
       [0.00488964],
       [0.01614396]])

In [16]:
#splitting dataset into train and test split
#training_size=int(len(dfe))
train_data=df1[:1500]
test_data=df1[1500:]

In [14]:
#len(train_data)

In [15]:
#len(test_data)

In [16]:
#function to create X,Y train and test dataset
def create_dataset(dataset,time_step):
    X=[]
    Y=[]
    for i in range(len(dataset)-time_step-1):
        a=dataset[i:i+time_step,0]
        X.append(a)
        Y.append(dataset[i+time_step,0])
    return np.array(X),np.array(Y)        

In [29]:
#creating train and test datasets
time_step=100
X_train,Y_train=create_dataset(train_data,time_step)
X_test,Y_test=create_dataset(test_data,time_step)

In [30]:
#X_train


In [31]:
#Y_train

In [32]:
#X_test

In [33]:
#Y_test

In [34]:
#X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

In [35]:
#plt.plot(X_train,Y_train)

In [36]:
# reshaping input into three dimensions :3 dimensions are=samples, time_steps,feature
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1) # adding 1 as one more dimension
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)# adding 1 as one more dimension


In [37]:
#creating a stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [38]:
#using Sequential LSTM model
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))#adding first layer of LSTM , giving two iputs :time_steps,1
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100, 50)           10400     
                                                                 
 lstm_4 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [40]:
#training LSTM model by giving train inputs
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
22/22 [==============================] - 18s 357ms/step - loss: 0.0011 - val_loss: 4.1859e-04
Epoch 2/100
22/22 [==============================] - 5s 245ms/step - loss: 0.0010 - val_loss: 4.2314e-04
Epoch 3/100
22/22 [==============================] - 5s 235ms/step - loss: 0.0010 - val_loss: 4.0113e-04
Epoch 4/100
22/22 [==============================] - 5s 227ms/step - loss: 0.0010 - val_loss: 4.0920e-04
Epoch 5/100
22/22 [==============================] - 5s 226ms/step - loss: 0.0010 - val_loss: 3.9802e-04
Epoch 6/100
22/22 [==============================] - 5s 230ms/step - loss: 0.0010 - val_loss: 4.1501e-04
Epoch 7/100
22/22 [==============================] - 6s 293ms/step - loss: 0.0010 - val_loss: 4.1001e-04
Epoch 8/100
22/22 [==============================] - 6s 272ms/step - loss: 0.0010 - val_loss: 4.2726e-04
Epoch 9/100
22/22 [==============================] - 6s 260ms/step - loss: 0.0010 - val_loss: 3.9904e-04
Epoch 10/100
22/22 [==============================] - 

KeyboardInterrupt: 

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score
regg=LinearRegression()

In [23]:
X_reg_train=train_data[:1500]
Y_reg_train=train_data[1500:]
regg.fit(X_reg_train,Y_reg_train)

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by LinearRegression.

In [ ]:
import tensorflow as tf

In [ ]:
#predicting values using created LSTM model
Y_predicted=model.predict(X_test)

In [ ]:
#transforming values back to original form
Y_predicted=scaler.inverse_transform(Y_predicted)

In [ ]:
import math #to find squreroot using sqrt function
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(Y_test,Y_predicted))

In [ ]:
#plotting values
plt.plot(scaler.inverse_transform(df1))
#plot(X_train,Y_train)
#plt.plot(X_test,Y_test)
#plt.plot(X_test,Y_predicted)
plt.show()